In [ ]:
import boto3
from dotenv import load_dotenv

# Load environment variables from .env
load_dotenv()

# boto3 will automatically pick up credentials from environment variables
s3 = boto3.client("s3")
bucket = 'fypwhere'
before_key = 'eu_cookie_new.pdf'
after_key = 'eu_cookie_old.pdf'

before_url = s3.generate_presigned_url(
    ClientMethod="get_object",
    Params={"Bucket": bucket, "Key": before_key},
    ExpiresIn=100
)

after_url = s3.generate_presigned_url(
    ClientMethod="get_object",
    Params={"Bucket": bucket, "Key": after_key},
    ExpiresIn=100
)

print("Generated before presigned URL:", before_url)
print("Generated after presigned URL:", after_url)



In [3]:

from dotenv import load_dotenv
import os

load_dotenv()  # loads variables from .env

perplexity_key = os.getenv("OPENAI_API_KEY")

from langchain_core.prompts import ChatPromptTemplate
from langchain_perplexity import ChatPerplexity

chat = ChatPerplexity(temperature=0, model="sonar") 

system = "You explain regulatory changes clearly and concisely. Rely only on the provided “Before” and “After” documents. Quote exact text with locations when evidencing changes."
human = (
   """User:
Compare the regulation Before vs After at these links:

Before: {before_url}

After: {after_url}

Output:
Produce a list of changes, each with:

summary: one sentence in plain language.

analysis: 2–4 sentences on implications, scope, and who is affected.

change: precise description (e.g., thresholds, deadlines, definitions, scope, penalties, references).

before_quote: exact excerpt + location (page/section).

after_quote: exact excerpt + location (page/section).

type: addition | deletion | modification | renumbering | scope change | threshold change | definition change | reference update | timeline change | penalty change | procedural change.

confidence: 0.00–1.00.

Method:

Extract and align sections by titles/numbering; note renumbering if applicable.

For each aligned or unmatched section, detect substantive edits; ignore purely cosmetic edits.

Prefer 2–6 sentence evidence quotes per side.

Also include:

Unchanged sections: list ids/titles with brief reason.

Potentially related edits: cross-references, global metadata (title, effective date, jurisdiction).

Format:

Human-readable report with “summary / analysis / change” for every detected change.

Then a JSON array containing those same fields for each change entry."""
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    ("human", human)
])

chain = prompt | chat

response = chain.invoke({
    "before_url": before_url,
    "after_url": after_url
})

print(response.content)


Here is a detailed comparison report of the EU cookie regulation documents titled "Before" (new version) and "After" (old version) based on the provided PDFs. The analysis aligns sections by titles and numbering, identifies substantive changes, and provides exact quotes with locations.

---

### Change 1  
**Summary:** The new regulation tightens the definition and scope of consent, emphasizing explicit, informed, and prior consent before any non-essential cookies are set.  
**Analysis:** This change increases the burden on website operators to obtain clear, unambiguous consent before placing cookies that are not strictly necessary. It affects all data controllers and processors operating in the EU, requiring more transparent consent mechanisms and stricter compliance to avoid penalties. This aligns with the 2025 enforcement trend toward penalizing dark patterns and pre-ticked boxes.  
**Change:** The new regulation explicitly requires "prior consent" and forbids any non-essential cook

In [ ]:
from langchain_ollama.llms import OllamaLLM

chat = OllamaLLM(model = "mistral")

system = "You are a clear and concise assistant who explains regulatory changes."
human = (
    "Explain the change in the regulation about {regulation_topic}.\n"
    "The regulation before the change was:\n{before_text}\n\n"
    "The regulation after the change is:\n{after_text}\n\n"
    "Please describe what changed."
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    ("human", human)
])

chain = prompt | chat

response = chain.invoke({
    "regulation_topic": "data privacy",
    "before_text": "Personal data could be stored indefinitely without user consent.",
    "after_text": "Personal data must be deleted after 2 years unless consent is renewed."
})

print(response)
